In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import os
import cv2

import sys
sys.path.append('..')

from shared.data import KITTIData, draw_matches

%matplotlib widget

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
# https://github.com/matplotlib/ipympl

In [ ]:
DATASET_DIR = os.path.join('../', 'data/KITTI/dataset')
dataset = KITTIData(DATASET_DIR, sequence_id="00")

# Test stereovision

In [ ]:
l_img, r_img = dataset.get_images(30)
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=[14,10])
ax1.imshow(l_img)
ax2.imshow(r_img)

In [ ]:
block_sz = 5
left_matcher = cv2.StereoBM_create(numDisparities=16*4, blockSize=block_sz)

# wls_filter = cv2.ximgproc.createDisparityWLSFilter(left_matcher)
# right_matcher = cv2.ximgproc.createRightMatcher(left_matcher)

left_matcher.setMinDisparity(0)
left_matcher.setSpeckleRange(4)
left_matcher.setSpeckleWindowSize(100)
# left_matcher.setDisp12MaxDiff(1)
left_matcher.setUniquenessRatio(10)
left_matcher.setPreFilterCap(31)
left_matcher.setPreFilterSize(9)
left_matcher.setTextureThreshold(10)

l_disp = left_matcher.compute(l_img, r_img)
# r_disp = right_matcher.compute(r_img_gray, l_img_gray)
disp = l_disp.astype(np.float32) / 16.0

# wls_filter.setLambda(8000)
# wls_filter.setSigmaColor(8000)
# # wls_filter->filter(left_disp,left,filtered_disp,right_disp);
# wls_filter.filter(l_disp, l_img_gray, )
plt.figure(figsize=[14,5])
plt.imshow(disp)

In [ ]:
block_sz = 3
left_matcher = cv2.StereoSGBM_create(
    minDisparity=0,
    numDisparities=16*10, 
    blockSize=block_sz,
    P1=block_sz*block_sz*8*3,
    P2=block_sz*block_sz*32*3,
#     disp12MaxDiff=0,
    preFilterCap=63,
    uniquenessRatio=10,
    speckleWindowSize=100,
    speckleRange=1,
#     mode=cv2.StereoSGBM_MODE_SGBM_3WAY
)

l_disp = left_matcher.compute(l_img, r_img)
disp = l_disp.astype(np.float32) / 16.0
plt.figure(figsize=[14,5])
plt.imshow(disp)

In [ ]:
Q_left = dataset.get_left_Q_matrix()
print(Q_left)
points = cv2.reprojectImageTo3D(disp, Q_left)

plt.figure(figsize=(15,9))
plt.imshow(points[:,:,2])

# points = points.reshape(-1, 3)
idxs = (disp > 0) & (points[:,:,2] < 100)
points = points[idxs]
colors = l_img[idxs]

print(colors.shape)
print(points.shape)

# colors = ['#%02x%02x%02x' % (clr[0], clr[1], clr[2]) for clr in colors]